# Базовый Python
# Проект: Музыка больших городов

## Описание проекта

Сравнение Москвы и Петербурга окружено мифами:
 - Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 - Петербург — город своеобразной культуры, непохожий на Москву.

Некоторые мифы отражают действительность. Другие — пустые стереотипы. Бизнес должен отличать первые от вторых, чтобы принимать рациональные решения. На реальных данных Яндекс Музыки вы проверите гипотезы и сравните поведение пользователей двух столиц.

### Гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. Утром в понедельник в Москве преобладают одни жанры музыки, а в Петербурге — другие. Это верно и для вечера пятницы.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

### Данные 

Данные хранятся в файле `yandex_music_project.csv`. Скачать датасет можно здесь.

**Описание колонок:**
 - `userID` — идентификатор пользователя;
 - `Track` — название трека;
 - `artist` — имя исполнителя;
 - `genre` — название жанра;
 - `City` — город пользователя;
 - `time` — время начала прослушивания;
 - `Day` — день недели.

### Ревью проекта

Вашу работу проверит ревьюер — опытный аналитик. Он поможет улучшить проект.
Для этого ревьюер оценит:
результаты каждого задания,
соблюдение правил синтаксиса Python,
стиль кода.

В течение суток ревьер вернётся с обратной связью. Например, предложит упростить сложную конструкцию в коде. Тогда вы сможете доработать проект и отправить ревьюеру обновлённый вариант.

Проект завершён, когда засчитаны все исправления.

Обратите внимание, что при запуске ячеек тетрадки в произвольном порядке вы можете столкнуться с ошибками. Например, в том случае, если попытаетесь получить объект, который в данный момент не загружен в память. Прежде чем обращаться к ревьюеру или преподавателю, попробуйте выгрузить все объекты из памяти и перезапустить весь код. Сделать это можно в Jupyter Notebook с помощью команды `Kernel` → `Restart & Run All`.

In [ ]:
Добро пожаловать в самостоятельный проект
Самостоятельный проект — это практическая проверка знаний, приобретённых вами на курсе. Каждый раздел посвящён отдельной стадии анализа данных с экскурсом в основы Python. Ещё эту работу можно добавить в портфолио. Вы начинаете собирать это портфолио прямо сейчас.
Проект выполняется в пять этапов:
Постановка задачи
Получение данных
Предобработка данных
Анализ данных
Оформление результатов
Для каждой части описаны шаги выполнения c теоретическим приложением. В Jupyter Notebook эти шаги связаны между собой выводами и результатами.
Вы закрепите применение операторов и методов языка Python (и его библиотеки Pandas) на разных стадиях анализа данных. Кроме того, получите первый опыт оформления в Jupyter Notebook проекта, которым можно поделиться.
Если возникнут сложности, всегда можно воспользоваться навигацией по пройденным урокам.